<a href="https://colab.research.google.com/github/faroukabdelmagied/House-Prices---Advanced-Regression-Techniques/blob/main/Part_4_Feature_Selection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Feature Selection

## Importing the Libraries

In [ ]:
## Importing the relevant packages

import pandas as pd
import numpy as np
import plotly.express as px

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder, RobustScaler, StandardScaler
from sklearn.compose import ColumnTransformer
from imblearn.pipeline import make_pipeline, Pipeline


# Import statsmodels for regression

import statsmodels.api as sm


/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning:

The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning:

The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning:

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.



In [ ]:
## Setting the Random Seed - to gurantee reproducibility:
np.random.seed(0)

## Importing the Enigneered Datasets

In [ ]:
## Reading the dataset:
df = pd.read_csv('X_train_featured.csv')

In [ ]:
## Exploring the dataframe
df
## Exploring the information of the dataset:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 84 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Id              1460 non-null   int64  
 1   MSSubClass      1460 non-null   object 
 2   MSZoning        1460 non-null   object 
 3   LotFrontage     1460 non-null   float64
 4   LotArea         1460 non-null   float64
 5   Street          1460 non-null   object 
 6   Alley           1460 non-null   object 
 7   LotShape        1460 non-null   object 
 8   LandContour     1460 non-null   object 
 9   Utilities       1460 non-null   object 
 10  LotConfig       1460 non-null   object 
 11  LandSlope       1460 non-null   object 
 12  Neighborhood    1460 non-null   object 
 13  Condition1      1460 non-null   object 
 14  Condition2      1460 non-null   object 
 15  BldgType        1460 non-null   object 
 16  HouseStyle      1460 non-null   object 
 17  OverallQual     1460 non-null   i

## Splitting the Dataset into X and y

In [ ]:
X = df.drop(columns=['SalePrice', 'Id'], axis=1)
y = df[['SalePrice']]

In [ ]:
## Checking the correctivness of the dataset:
y

,SalePrice
0,12.247694
1,12.109011
2,12.317167
3,11.849398
4,12.429216
...,...
1455,12.072541
1456,12.254863
1457,12.493130
1458,11.864462


## Encoding the Dataset

In [ ]:
## Encoding the categorical features using get_dummies
X_encoded = pd.get_dummies(X, drop_first=True)
X_encoded

,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,LotFrontagenan,MasVnrAreanan,GarageYrBltnan,MSSubClass_B,MSSubClass_E,...,KitchenQual_TA,Functional_Min2,Functional_Mod,Functional_Rare_var,Functional_Typ,FireplaceQu_Fa,FireplaceQu_Gd,FireplaceQu_Missing,FireplaceQu_Po,FireplaceQu_TA,GarageType_Basment,GarageType_BuiltIn,GarageType_Detchd,GarageType_Missing,GarageType_Rare_var,GarageFinish_Missing,GarageFinish_RFn,GarageFinish_Unf,GarageQual_Missing,GarageQual_Rare_var,GarageQual_TA,GarageCond_Missing,GarageCond_Rare_var,GarageCond_TA,PavedDrive_P,PavedDrive_Y,PoolQC_Rare_var,Fence_GdWo,Fence_Missing,Fence_MnPrv,Fence_Rare_var,MiscFeature_Rare_var,MiscFeature_Shed,SaleType_New,SaleType_Rare_var,SaleType_WD,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,SaleCondition_Rare_var
0,4.174387,9.041922,7,5,5,5,196.0,706,0,150,856,6.752270,854,0,7.444249,1,0,2,1,3,1,8,0,5.0,2,548,0,61,0,0,0,0,0,2,2008,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,0,1,0,1,0,0
1,4.382027,9.169518,6,8,31,31,0.0,978,0,284,1262,7.140453,0,0,7.140453,0,1,2,0,3,1,6,1,31.0,2,460,298,0,0,0,0,0,0,5,2007,0,0,0,0,0,...,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,0,1,0,1,0,0
2,4.219508,9.328123,7,5,7,6,162.0,486,0,434,920,6.824374,866,0,7.487734,1,0,2,1,3,1,6,1,7.0,2,608,0,42,0,0,0,0,0,9,2008,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,0,1,0,1,0,0
3,4.094345,9.164296,7,5,91,36,0.0,216,0,540,756,6.867974,756,0,7.448334,1,0,1,0,3,1,7,1,8.0,3,642,0,35,272,0,0,0,0,2,2006,0,0,0,0,0,...,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0
4,4.430817,9.565214,8,5,8,8,350.0,655,0,490,1145,7.043160,1053,0,7.695303,1,0,2,1,4,1,9,1,8.0,3,836,192,84,0,0,0,0,0,12,2008,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,4.127134,8.976768,6,5,8,7,0.0,0,0,953,953,6.859615,694,0,7.406711,0,0,2,1,3,1,7,1,8.0,2,460,0,40,0,0,0,0,0,8,2007,0,0,0,0,0,...,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,0,1,0,1,0,0
1456,4.442651,9.486076,6,6,32,22,119.0,790,163,589,1542,7.636752,0,0,7.636752,1,0,2,0,3,1,7,2,32.0,2,500,349,0,0,0,0,0,0,2,2010,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,1,0,0,0,1,0,0,0,0,0,1,0,1,0,0
1457,4.189655,9.109636,7,9,69,4,0.0,275,0,877,1152,7.080026,1152,0,7.757906,0,0,2,0,4,1,9,2,69.0,1,252,0,60,0,0,0,0,2500,5,2010,0,0,0,0,0,...,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,1,0,0,1,0,1,0,0
1458,4.219508,9.181632,5,6,60,14,0.0,49,1029,0,1078,6.982863,0,0,6.982863,1,0,1,0,2,1,5,0,60.0,1,240,366,0,112,0,0,0,0,4,2010,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,0,1,0,1,0,0


## Creating Train and Test set

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=0)
X_train

,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,LotFrontagenan,MasVnrAreanan,GarageYrBltnan,MSSubClass_B,MSSubClass_E,...,KitchenQual_TA,Functional_Min2,Functional_Mod,Functional_Rare_var,Functional_Typ,FireplaceQu_Fa,FireplaceQu_Gd,FireplaceQu_Missing,FireplaceQu_Po,FireplaceQu_TA,GarageType_Basment,GarageType_BuiltIn,GarageType_Detchd,GarageType_Missing,GarageType_Rare_var,GarageFinish_Missing,GarageFinish_RFn,GarageFinish_Unf,GarageQual_Missing,GarageQual_Rare_var,GarageQual_TA,GarageCond_Missing,GarageCond_Rare_var,GarageCond_TA,PavedDrive_P,PavedDrive_Y,PoolQC_Rare_var,Fence_GdWo,Fence_Missing,Fence_MnPrv,Fence_Rare_var,MiscFeature_Rare_var,MiscFeature_Shed,SaleType_New,SaleType_Rare_var,SaleType_WD,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,SaleCondition_Rare_var
618,4.499810,9.366831,9,5,0,0,452.0,48,0,1774,1822,7.510978,0,0,7.510978,0,0,2,0,3,1,9,1,0.0,3,774,0,108,0,0,260,0,0,7,2007,0,0,0,0,0,...,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0
870,4.094345,8.794825,5,5,47,47,0.0,0,0,894,894,6.795706,0,0,6.795706,0,0,1,0,2,1,5,0,47.0,1,308,0,0,0,0,0,0,0,8,2009,0,0,0,0,0,...,1,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,0,1,0,1,0,0
92,4.382027,9.500020,5,7,88,3,0.0,713,0,163,876,6.871091,0,0,6.871091,1,0,1,0,2,1,5,0,88.0,2,432,0,0,44,0,0,0,0,8,2009,0,0,0,1,0,...,1,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,0,1,0,1,0,0
817,4.234107,9.492884,8,5,6,6,148.0,1218,0,350,1568,7.431892,0,0,7.431892,1,0,2,0,3,1,7,2,6.0,3,857,150,59,0,0,0,0,0,7,2008,1,0,0,0,0,...,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,0,1,0,1,0,0
302,4.770685,9.525443,7,5,5,4,150.0,0,0,1541,1541,7.340187,0,0,7.340187,0,0,2,0,3,1,6,1,5.0,3,843,468,81,0,0,0,0,0,1,2006,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
763,4.406719,9.151651,8,5,10,10,673.0,1163,0,89,1252,7.145196,1097,0,7.768533,1,0,2,1,3,1,8,1,10.0,3,856,0,128,0,0,180,0,0,7,2009,0,0,0,0,0,...,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,0,1,0,1,0,0
835,4.094345,9.169518,4,7,60,15,0.0,442,0,625,1067,6.972606,0,0,6.972606,0,0,2,0,2,1,4,0,14.0,2,436,290,0,0,0,0,0,0,2,2010,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,0,1,0,1,0,0
1216,4.219508,9.097172,6,5,32,32,0.0,0,0,0,0,7.183871,584,0,7.550661,0,0,2,0,4,2,8,0,32.0,2,539,0,0,0,0,0,0,0,4,2010,0,0,0,0,0,...,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,0,1,0,1,0,0
559,4.234107,8.069655,7,5,3,2,18.0,0,0,1374,1374,7.350516,0,0,7.350516,0,0,2,0,2,1,7,1,3.0,2,420,143,20,0,0,0,0,0,10,2006,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,0,1,0,1,0,0


# Feature Selection Process

## Applying Lasso Regression to identify the most relevant features

In [ ]:
## Importing the needed libraries
from sklearn.linear_model import Lasso
from sklearn.feature_selection import SelectFromModel

In [ ]:
## Apply Feature Selection using the Lasso Regression model, the bigger the alpha the less features will be selected:
## Then we will use the SelectFromModel object from sklearn, which will select the features which coefficients are
# no-zero:
feature_sel_model=SelectFromModel(Lasso(alpha=0.005, random_state=0)) ## Setting the seed to get same results again
feature_sel_model.fit(X_train,y_train)


SelectFromModel(estimator=Lasso(alpha=0.005, copy_X=True, fit_intercept=True,
                                max_iter=1000, normalize=False, positive=False,
                                precompute=False, random_state=0,
                                selection='cyclic', tol=0.0001,
                                warm_start=False),
                max_features=None, norm_order=1, prefit=False, threshold=None)

In [ ]:
## Selected Features:
feature_sel_model.get_support()

array([False,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True, False, False, False, False,
       False, False, False,  True,  True,  True, False,  True,  True,
        True,  True,  True,  True,  True, False,  True, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False,  True, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False,  True, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,

In [ ]:
## Let us print the number of selected features

## This is how we can make a list of all the selected features
selected_feat = X_train.columns[(feature_sel_model.get_support())]

## Lets print some stats:
print('total features: {}'.format((X_train.shape[1])))
print('selected features: {}'.format(len(selected_feat)))

total features: 220
selected features: 30


In [ ]:
## Print out the selected features:
selected_feat

Index(['LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF',
       '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'TotRmsAbvGrd', 'Fireplaces',
       'GarageYrBlt', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF',
       'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MoSold',
       'MSZoning_RL', 'Condition1_Norm', 'Foundation_CBlock', 'HeatingQC_TA',
       'KitchenQual_TA', 'FireplaceQu_Gd'],
      dtype='object')

## New Data Set with relevant features

In [ ]:
Dataset = X_train[selected_feat]

In [ ]:
Dataset

,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MoSold,MSZoning_RL,Condition1_Norm,Foundation_CBlock,HeatingQC_TA,KitchenQual_TA,FireplaceQu_Gd
618,9.366831,9,5,0,0,452.0,48,0,1774,1822,7.510978,0,0,9,1,0.0,774,0,108,0,0,260,0,7,1,1,0,0,0,1
870,8.794825,5,5,47,47,0.0,0,0,894,894,6.795706,0,0,5,0,47.0,308,0,0,0,0,0,0,8,1,0,1,0,1,0
92,9.500020,5,7,88,3,0.0,713,0,163,876,6.871091,0,0,5,0,88.0,432,0,0,44,0,0,0,8,1,1,0,0,1,0
817,9.492884,8,5,6,6,148.0,1218,0,350,1568,7.431892,0,0,7,2,6.0,857,150,59,0,0,0,0,7,1,1,0,0,0,1
302,9.525443,7,5,5,4,150.0,0,0,1541,1541,7.340187,0,0,6,1,5.0,843,468,81,0,0,0,0,1,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
763,9.151651,8,5,10,10,673.0,1163,0,89,1252,7.145196,1097,0,8,1,10.0,856,0,128,0,0,180,0,7,1,1,0,0,0,1
835,9.169518,4,7,60,15,0.0,442,0,625,1067,6.972606,0,0,4,0,14.0,436,290,0,0,0,0,0,2,1,1,1,1,0,0
1216,9.097172,6,5,32,32,0.0,0,0,0,0,7.183871,584,0,8,0,32.0,539,0,0,0,0,0,0,4,0,0,0,1,1,0
559,8.069655,7,5,3,2,18.0,0,0,1374,1374,7.350516,0,0,7,1,3.0,420,143,20,0,0,0,0,10,1,1,0,0,0,0


In [ ]:
## Saving the data set:
Dataset.to_csv('Selected_Features_set.csv', index = False)